# Configuração Inicial

# PuLP Model

In [16]:
import pulp as pl

In [17]:
def build_model(name: str, n: int, m: int, cap: list[int], f: list[int],
                d: list[int], c: list[list[int]], relaxed: bool=False,
                modified: bool=False) -> pl.LpProblem:

    if modified:
      model_type = '-m'
    elif relaxed:
      model_type = '-r'
    else:
      model_type = '-p'
    name += model_type

    model = pl.LpProblem(f"{name}-Location-Of-Facilities", pl.LpMinimize)

    facilities = range(n)
    clients = range(m)

    y = {}
    if relaxed or modified:
        for i in facilities:
            y[i] = pl.LpVariable(f"y_{i}", lowBound=0, upBound=1,
                                 cat=pl.LpContinuous)
    else:
        for i in facilities:
            y[i] = pl.LpVariable(f"y_{i}", cat=pl.LpBinary)

    x = {}
    for i in facilities:
        for j in clients:
            x[i, j] = pl.LpVariable(f"x_{i}_{j}", lowBound=0, upBound=1,
                                    cat=pl.LpContinuous)

    for j in clients:
        model += pl.lpSum(x[i, j] for i in facilities) == 1, f"Restrição_2_{j}"

    if modified:
        for i in facilities:
            model += pl.lpSum(d[j] * x[i, j] for j in clients) <= cap[i],\
            f"Restrição_3_nova_{i}"
            for j in clients:
                model += x[i, j] <= y[i], f"Restrição_11_{i}_{j}"
    else:
        for i in facilities:
            model += pl.lpSum(d[j] * x[i, j] for j in clients) <= cap[i] * y[i], \
                     f"Restrição_3_{i}"

    # Construir a função objetivo usando lpSum
    model += pl.lpSum(f[i] * y[i] for i in facilities) + \
             pl.lpSum(c[j][i] * x[i, j] for i in facilities for j in clients)

    return model

# Leitura das Instâncias

In [18]:
def read_instance(instance_name: str) -> tuple[int, int, list[int], list[int],
                                                list[int], list[list[int]]]:
  with open(f"{instance_name}.txt") as reader:
    n, m = map(int, reader.readline().split())

    cap = []
    f = []
    for _ in range(n):
      cap_i, f_i = map(int, reader.readline().split())
      cap.append(cap_i)
      f.append(f_i)

    d = []
    c = []
    for _ in range(m):
      d_i, *c_i = map(int, reader.readline().split())
      d.append(d_i)
      c.append(c_i)

  return n, m, cap, f, d, c

# Soluções

In [19]:
import gurobipy as gp

In [20]:
def solve_gurobi(model: pl.LpProblem, time_limit=300) -> str:
  options = {
      "WLSACCESSID": '5b92c70a-c10f-44dd-866e-19a6a2fb9372',
      "WLSSECRET": '111e2b19-d811-4b2f-8ef7-18f1f12e03c1',
      "LICENSEID": 2443615
  }

  with gp.Env(params=options) as env:
    # Pass environment as a parameter
    filename = f"{model.name}-GUROBI.txt"
    solver = pl.GUROBI(timeLimit=time_limit, logPath=filename,
                       LogToConsole=False)
    model.solve(solver)

    return filename


def solve_scip(model: pl.LpProblem, time_limit=300) -> str:
    filename = f"{model.name}-SCIP.txt"
    solver = pl.SCIP_PY(timeLimit=time_limit, msg=True) # ou SCIP_CMD ?
    model.solve(solver)

    return filename

## Geração dos Arquivos de Log

In [7]:
instances = ["wlp01", "wlp02", "wlp03", "wlp04", "wlp05"]

### Modelo Padrão

In [22]:
gurobi_p_log_files = []
scip_p_log_files = []

# Geração dos log_files do modelo no formato padrão
for instance in instances:
    model = build_model(instance, *read_instance(instance))
    gurobi_p_log_files.append(solve_gurobi(model))
    scip_p_log_files.append(solve_scip(model))

Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2443615
Academic license 2443615 - for non-commercial use only - registered to th___@gmail.com

Interrupt request received
Gurobi status= 11


In [10]:
for instance in instances:
    model = build_model(instance, *read_instance(instance))
    result = solve_scip(model)

### Modelo Relaxado

In [ ]:
# Geração dos log_files do modelo no formato relaxado
gurobi_r_log_files = []

for instance in instances:
    model = build_model(instance, *read_instance(instance), relaxed=True)
    gurobi_r_log_files.append(solve_gurobi(model))

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2443615
Academic license 2443615 - for non-commercial use only - registered to th___@gmail.com
Gurobi status= 2


### Modelo Relaxado e Modificado

In [11]:
# Geração dos log_files do modelo no formato relaxado e modificado
gurobi_r_m_log_files = []

for instance in instances:
    model = build_model(instance, *read_instance(instance), \
                        relaxed=True, modified=True)
    gurobi_r_m_log_files.append(solve_gurobi(model))

Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2443615
Academic license 2443615 - for non-commercial use only - registered to th___@gmail.com
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-24
Gurobi status= 2
Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2443615
Academic license 2443615 - for non-commercial use only - registered to th___@gmail.com
Gurobi status= 2
Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2443615
Academic license 2443615 - for non-commercial use only - registered to th___@gmail.com
Gurobi status= 2
Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2443615
Academic license 2443615 - for non-commercial use only - registered to th___@gmail.com
Gurobi status= 2
Set parameter Username
Set parameter 

# Coleta dos Dados

In [1]:
import orloge as ol
import re

In [38]:
def extract_infos_scip_log(log_file: str):

  with open('logs/'+log_file, 'r') as file:
      content = file.read()

  content = content.split('\n')
  pattern = r"[-+]?\d*\.\d+[eE][-+]?\d+"
  try:
    status = content[-7].split(':')[1].strip()
    exec_time = float(content[-6].split(':')[1])
    primal_bound = float(re.findall(pattern, content[-4])[0])
    dual_bound = float(re.findall(pattern, content[-3])[0])
    gap = float(content[-2].split(':')[1].rstrip('%'))
  except:
    status = content[-5].split(':')[1].strip()
    exec_time = float(content[-4].split(':')[1])
    primal_bound = '-'
    dual_bound = '-'
    gap = '-'
    
  return { log_file.split('-')[0]:
           {
              'solver':'Scip',
              'status':status,
              'exec_time':exec_time,
              'primal_bound':primal_bound,
              'dual_bound': dual_bound,
              'gap': gap
           }
          }

In [44]:
def extract_infos_gurobi_log(log_file: str):
    out = ol.get_info_solver('logs/'+log_file, 'GUROBI')
    name = log_file.split('-')[0]

    optimal_objective = 0
    exec_time = 0
    log = ''
    if out['status'] == None:
        with open('logs/'+log_file, 'r') as file:
            log = file.read()

        op = re.compile(r'Optimal objective\s+(\d.+)')
        tp = re.compile(r'Solved in \d+ iterations and ([\d.]+) seconds')
        objective = op.search(log)
        time = tp.search(log)
        optimal_objective = float(objective.group(1)) if objective else None
        exec_time = float(time.group(1)) if time else None

    # Time limit / Non result
    if log != '' and optimal_objective == None:
        tp = re.compile(r'Stopped in \d+ iterations and ([\d.]+) seconds')
        time = tp.search(log)
        exec_time = float(time.group(1)) if time else None

        return { name:
                {
                    'status':'Time limit reached',
                    'solver':'Gurobi',
                    'exec_time':exec_time,
                    'primal_bound':'-',
                    'dual_bound': '-',
                    'gap': '-'
                }
            }
    # Optimal 
    elif log != '' and optimal_objective:
        return { name:
                {
                    'status':'Optimal objective',
                    'solver':'Gurobi',
                    'exec_time':exec_time,
                    'primal_bound':optimal_objective,
                    'dual_bound': optimal_objective,
                    'gap': 0.00
                }
            }
    # 
    else:
        return { name:
                {
                    'status':out['status'],
                    'solver':'Gurobi',
                    'exec_time':out['time'],
                    'primal_bound':out['best_bound'],
                    'dual_bound': out['best_solution'],
                    'gap': out['gap']
                }
            }

## Execuções

In [4]:
import pandas as pd
from math import isnan

In [5]:
c = ['Solver', 'Instance', 'Status', 'Time Executing', 'Primal Value', 'Dual Value', 'Gap']

### SCIP Padrão

In [40]:
name_scip_p_files = [name+'-p-Location-Of-Facilities-SCIP.txt' for name in instances]

log_results_scip_p = {}
for log_file in name_scip_p_files:
  log_results_scip_p.update(extract_infos_scip_log(log_file))
  
df_scip_p = pd.DataFrame(columns=c)

for key, value in log_results_scip_p.items():
  new_index = df_scip_p.index.max() + 1  # Calcula o novo índice
  if isnan(new_index):
      new_index = 0
        
  df_scip_p.loc[new_index] = pd.Series({ # type: ignore
      'Solver': value['solver'],
      'Instance': key,
      'Status': value['status'],
      'Time Executing': value['exec_time'],
      'Primal Value': value['primal_bound'],
      'Dual Value': value['dual_bound'],
      'Gap': value['gap']
  })

df_scip_p.tail()

,Solver,Instance,Status,Time Executing,Primal Value,Dual Value,Gap
0,Scip,wlp01,problem is solved [optimal solution found],270.06,69055.247022,69055.247022,0.0
1,Scip,wlp02,solving was interrupted [time limit reached],300.01,75932.681862,75885.30277,0.06
2,Scip,wlp03,solving was interrupted [time limit reached],329.37,114899.854105,114676.150259,0.2
3,Scip,wlp04,solving was interrupted [time limit reached],300.2,135502.376356,135101.303795,0.3
4,Scip,wlp05,solving was interrupted [time limit reached],348.42,-,-,-


### Gurobi Padrão

In [42]:
name_gurobi_p_files = [name+'-p-Location-Of-Facilities-GUROBI.txt' for name in instances]

log_results_gurobi_p = {}
for log_file in name_gurobi_p_files:
  log_results_gurobi_p.update(extract_infos_gurobi_log(log_file))
  
df_gurobi_p = pd.DataFrame(columns=c)

for key, value in log_results_gurobi_p.items():
    new_index = df_gurobi_p.index.max() + 1  # Calcula o novo índice
    if isnan(new_index):
        new_index = 0
        
    df_gurobi_p.loc[new_index] = pd.Series({ # type: ignore
        'Solver': value['solver'],
        'Instance': key,
        'Status': value['status'],
        'Time Executing': value['exec_time'],
        'Primal Value': value['primal_bound'],
        'Dual Value': value['dual_bound'],
        'Gap': value['gap']
    })

df_gurobi_p.tail()

,Solver,Instance,Status,Time Executing,Primal Value,Dual Value,Gap
0,Gurobi,wlp01,Optimal solution found,4.68,69049.491081,69055.247022,0
1,Gurobi,wlp02,Optimal solution found,39.23,75931.138805,75934.725152,0
2,Gurobi,wlp03,Optimal solution found,206.3,114736.905754,114736.905754,0
3,Gurobi,wlp04,Time limit reached,300.07,135110.770067,135143.455855,0.0242
4,Gurobi,wlp05,Time limit reached,300.09,161960.003551,162022.527208,0.0386


### Gurobi relaxado

In [45]:
name_gurobi_r_files = [name+'-r-Location-Of-Facilities-GUROBI.txt' for name in instances]

log_results_gurobi_r = {}
for log_file in name_gurobi_r_files:
  log_results_gurobi_r.update(extract_infos_gurobi_log(log_file))
  
df_gurobi_r = pd.DataFrame(columns=c)

for key, value in log_results_gurobi_r.items():
    new_index = df_gurobi_r.index.max() + 1  # Calcula o novo índice
    if isnan(new_index):
        new_index = 0
        
    df_gurobi_r.loc[new_index] = pd.Series({ # type: ignore
        'Solver': value['solver'],
        'Instance': key,
        'Status': value['status'],
        'Time Executing': value['exec_time'],
        'Primal Value': value['primal_bound'],
        'Dual Value': value['dual_bound'],
        'Gap': value['gap']
    })

df_gurobi_r.tail()

,Solver,Instance,Status,Time Executing,Primal Value,Dual Value,Gap
0,Gurobi,wlp01,Optimal objective,0.6,69005.35746,69005.35746,0.0
1,Gurobi,wlp02,Optimal objective,0.81,75840.63419,75840.63419,0.0
2,Gurobi,wlp03,Optimal objective,1.89,114633.6517,114633.6517,0.0
3,Gurobi,wlp04,Optimal objective,2.71,135069.0235,135069.0235,0.0
4,Gurobi,wlp05,Optimal objective,5.31,161910.9422,161910.9422,0.0


In [46]:
name_gurobi_m_files = [name+'-m-Location-Of-Facilities-GUROBI.txt' for name in instances]

log_results_gurobi_m = {}
for log_file in name_gurobi_m_files:
  log_results_gurobi_m.update(extract_infos_gurobi_log(log_file))
  
df_gurobi_m = pd.DataFrame(columns=c)

for key, value in log_results_gurobi_m.items():
    new_index = df_gurobi_m.index.max() + 1  # Calcula o novo índice
    if isnan(new_index):
        new_index = 0
        
    df_gurobi_m.loc[new_index] = pd.Series({ # type: ignore
        'Solver': value['solver'],
        'Instance': key,
        'Status': value['status'],
        'Time Executing': value['exec_time'],
        'Primal Value': value['primal_bound'],
        'Dual Value': value['dual_bound'],
        'Gap': value['gap']
    })

df_gurobi_m.tail()

,Solver,Instance,Status,Time Executing,Primal Value,Dual Value,Gap
0,Gurobi,wlp01,Optimal objective,14.63,12808.6301,12808.6301,0.0
1,Gurobi,wlp02,Optimal objective,29.53,14029.7966,14029.7966,0.0
2,Gurobi,wlp03,Optimal objective,83.83,18121.5379,18121.5379,0.0
3,Gurobi,wlp04,Optimal objective,165.82,20797.0848,20797.0848,0.0
4,Gurobi,wlp05,Optimal objective,222.07,22973.2161,22973.2161,0.0
